## Wczytywanie potrzebnych bibliotek.

In [27]:
import pandas as pd
import pathlib as path
import numpy as np
import datetime as dt
import calendar
from string import digits

# Gęstość rozkładu N(m, sd).
norm_dens : callable = lambda t, m, sd : 1/np.sqrt(2*np.pi * sd**2) * np.exp(-((t-m)/sd)**2)

digits:list[str] = list(digits)
today: dt.date = dt.date.today()


### Odczytywanie popularnych imion i nazwisk żeńskich i męskich.

In [28]:
# Znajdź katalog z imionami i nazwiskami.
name_dir_path: path.Path = path.Path().cwd()/"Imiona i nazwiska"

# Znajdź  katalog, gdzie zapisujemy wszystkie tabele .csv.
csv_tables_dir_path: path.Path = path.Path().cwd()/"Tabele_csv"


def read_names(first_names_file:str, last_names_file:str, dir_path:path.Path) -> tuple[pd.Series, pd.Series]:
    """Funkcja odczytuje pliki zawierające najpopularniejsze polskie imiona i nazwiska. Wczytuje je do szeregu danych typu pandas"""
    # Odczytaj imiona oraz nazwiska.
    first_names: pd.DataFrame = pd.read_excel(dir_path/first_names_file  ,"Arkusz1",
                                                usecols = [0,2]
                                                )
    last_names: pd.DataFrame = pd.read_excel(dir_path/last_names_file, "Arkusz1",
                                                nrows = 5_000) # Wierszy w arkuszu jest bardzo dużo (ponad 380 000). 
                                                                # Odczytaj tylko pewną część wierszy.
                                                                

    # Bierzemy pod uwagę jedynie najpopularniejsze imiona i nazwiska.
    first_names = first_names.loc[first_names["LICZBA_WYSTĄPIEŃ"] > 500, "IMIĘ_PIERWSZE"]
    last_names = last_names.loc[last_names["Liczba"] > 500, "Nazwisko aktualne"] 

    # Skróc nazwy kolumn.
    first_names.columns = ['imię']
    last_names.columns = ['nazwisko']

    return first_names, last_names

# Wczytaj męskie imiona i nazwiska.
male_first_names, male_last_names = read_names("imiona_meskie.xlsx", "nazwiska_meskie.xlsx", name_dir_path)

# Wczytaj żęńskie imiona i nazwiska.
female_first_names, female_last_names = read_names("imiona_zenskie.xlsx", "nazwiska_zenskie.xlsx", name_dir_path)


### Generujące funkcje pomocnicze.

In [56]:
def generate_birthday() -> dt.datetime:
    # Wygeneruj wiek z rozkładu normalnego. Pamiętaj, że osoba musi być przynajmniej pełnoletnia i maksymalnie 100 letnia.
    birth_year: int =  today.year - np.random.randint(18, 100,)

    birth_month:int = np.random.randint(1, 13) # Wygeneruj miesiąc urodzenia.

    birth_day = calendar.monthrange(birth_year, birth_month)[1] # Znajdź dzień urodzenia (zgodny z liczbą dni w miesiącu)

    birthday = dt.date(birth_year, birth_month, birth_day) # Złóż wszystkie składowe urodzin w jedną date.

    return birthday



def generate_email(first_name:str, last_name:str, domains:list[str]) -> str:
    email:str = first_name.lower() + last_name.lower() + "".join(np.random.choice(digits, 3)) +"@"+ np.random.choice(domains)

    return email



def generate_phone_number() -> str:
    # Wygeneruj losowy numer telefonu nie zaczynający się od 0.
    phone_number:np.array = np.random.choice(digits, 9)

    # Jeżeli wylosowaliśmy zero, podmień cyfrę.
    if phone_number[0] == "0":
        phone_number[0] = np.random.choice(digits[1:])

    phone_number: str = "".join(phone_number)

    return phone_number


def insert_id_col(people_data:pd.DataFrame, col_name:str) -> pd.DataFrame:
    people_data.insert(loc = 0,column = col_name,
                       value = range(1, people_data.shape[0]+1))
    
    return people_data


def merge_females_and_males(female_data:pd.DataFrame, male_data:pd.DataFrame, id_col_name:str) ->pd.DataFrame:
    people_data_df : pd.DataFrame = pd.concat(objs = [female_data, male_data], axis = 0).reset_index(drop = True)

    people_data_df : pd.DataFrame = insert_id_col(people_data_df, id_col_name)

    return people_data_df
    


### Tworzenie danych osobowych pracowników.

In [57]:
def generate_worker_personal_data(first_names:pd.Series, last_names:pd.Series, trades_n:dict[str,int], email_domains:str) -> pd.DataFrame:
    """Funkcja generuje dane osobiste pracowników. Tworzone są wiersze imię, nazwisko, numer telefonu oraz adres e-mail firmowy.

    """
    # Przygotuj wyjściową ramkę danych.
    workers_df: pd.DataFrame = pd.DataFrame(data = {"imię":pd.Series(dtype = pd.StringDtype()),
                                                    "nazwisko": pd.Series(dtype = pd.StringDtype()), 
                                                    "stanowisko":pd.Series(dtype = pd.CategoricalDtype(categories = ["kierowca","przewodnik"],
                                                                                                  ordered = False
                                                                                                  )),
                                                    "numer telefonu":pd.Series(dtype = pd.StringDtype()), 
                                                    "adres email": pd.Series(dtype = pd.StringDtype())},                                           
    )

    id: int = 0
    for pos in trades_n:
        for _ in range(trades_n[pos]):
            first_name_idx:int = np.random.randint(0, first_names.shape[0])
            last_name_idx:int = np.random.randint(0, last_names.shape[0])

            first_name:str = first_names[first_name_idx].capitalize() # Wylosuj imię
            last_name:str = last_names[last_name_idx].capitalize()  # Wylosuj nazwisko.

            # Wygeneruj losowy numer telefonu nie zaczynający się od 0.
            phone_number:np.array = np.random.choice(digits, 9)

            # Jeżeli wylosowaliśmy zero, podmień cyfrę.
            if phone_number[0] == "0":
                phone_number[0] = np.random.choice(digits[1:], 1)[0]

            phone_number: str = "".join(phone_number)

            # Stwórz email dla tego pracownika
            random_domain = np.random.choice(email_domains)
            email:str = first_name.lower() + last_name.lower() + "".join(np.random.choice(digits, 3)) +"@" + random_domain
            

            workers_df.loc[id, :] = [first_name, last_name, pos ,phone_number, email]

            id += 1

    return workers_df





# Dwa razy więcej przewodników niż mężczyzn.
female_drivers : int = 15
females_guides: int = 2*female_drivers

male_drivers:int = 25
male_guides = 2*male_drivers

female_trades_n: dict[str, int] = {"przewodnik":females_guides, 
                                   "kierowca":female_drivers}

male_trades_n: dict[str, int] = {"przewodnik":male_guides, 
                                 "kierowca":male_guides}


# Wygeneruj dane osobiste dla pracowniczek
email_domains:list[str] = ["nowowiejska.com"]

female_workers_df : pd.DataFrame = generate_worker_personal_data(female_first_names, female_last_names, female_trades_n, email_domains)

male_workers_df : pd.DataFrame = generate_worker_personal_data(male_first_names, male_last_names, male_trades_n, email_domains)

# Połącz kobiecych i męskich pracowników.
workers_data_df : pd.DataFrame = merge_females_and_males(female_workers_df, male_workers_df, "id_pracownika")


# Zapisz dane pracowników do pliku .csv.
workers_data_df.to_csv(path_or_buf = csv_tables_dir_path/"Pracownicy.csv",
                    sep = ";", index = False)



## Tworzenie danych osobowych klientów.

In [60]:
def generate_client_personal_data(first_names:pd.Series, last_names:pd.Series, n:int) -> pd.DataFrame:
    """Funkcja generuje dane osobiste klientów. Tworzone są wiersze imię, nazwisko, data urodzenia, numer telefonu i adres e-mail, status studenta.

    """
    # Przygotuj wyjściową ramkę danych.
    clients_df: pd.DataFrame = pd.DataFrame(data = {"imię":pd.Series(dtype = pd.StringDtype()),
                                                    "nazwisko": pd.Series(dtype = pd.StringDtype()), 
                                                    "data_urodzenia":pd.Series(dtype = "datetime64[ns]"),
                                                    "zniżka": pd.Series(dtype = pd.Float32Dtype()),
                                                    "numer telefonu":pd.Series(dtype = pd.StringDtype()), 
                                                    "adres email": pd.Series(dtype = pd.StringDtype())}                                          
    )

    clients_df["data_urodzenia"] = clients_df['data_urodzenia'].dt.date

    # Lista wszystkich cyfr. Przyda się przy generowaniu numerów telefonów.
    digits:list[str] = ["0","1","2","3","4","5","6","7","8","9"]
    # Lista domen emailowych.
    email_domains:list[str] = ['gmail.com', "wp.pl", "onet.pl"]



    for id in range(n):
        first_name_idx:int = np.random.randint(0, first_names.shape[0])
        last_name_idx:int = np.random.randint(0, last_names.shape[0])

        first_name:str = first_names[first_name_idx].capitalize() # Wylosuj imię
        last_name:str = last_names[last_name_idx].capitalize()  # Wylosuj nazwisko.

        
        # Wygeneruj losowy numer telefonu nie zaczynający się od 0.
        phone_number:str = generate_phone_number()

        # Stwórz email dla tego klienta
        email:str = generate_email(first_name, last_name, email_domains)
        
        
        # Zmyśl datę urodzenia dla tego klienta. Wszyscy muszą być pełnoletni i zakładamy, że żyją maksymalnie 100 lat.
        birthday = generate_birthday()
        

        age: int = (today-birthday).days/365

        # Ustal zniżkę w zależności od wieku.
        if age >= 70:
            discount = 0.5
        elif age >= 60:
            discount = 0.65
        elif age >= 50:
            discount = 0.75
        elif age >= 40:
            discount = 0.85
        elif age >= 30:
            discount = 0.9
        else:
            discount = 1
    

        clients_df.loc[id, :] = [first_name, last_name, birthday, discount,phone_number, email]

    return clients_df

        


n_female_clients: int = 50
n_male_clients:int = 76

female_clients_df : pd.DataFrame = generate_client_personal_data(female_first_names, female_last_names, n_female_clients)
male_clients_df : pd.DataFrame = generate_client_personal_data(male_first_names, male_last_names, n_male_clients)

clients_df = merge_females_and_males(female_clients_df, male_clients_df, "id_klienta")


clients_df.to_csv(path_or_buf = csv_tables_dir_path/"Klienci_db.csv",
                    sep = ";", index = False)


### Tworzenie tabeli Miasta oraz Atrakcje_w_miescie

In [81]:
all_info_trip = pd.read_csv(csv_tables_dir_path/"wycieczki.csv",
                              sep = ";", 
                              encoding = "ISO 8859-2").drop(columns = "LiczbaAtrakcji")


# Miasta do tabeli csv.
city_df : pd.DataFrame = all_info_trip[["Miasto"]]
city_df.loc[:, "Miasto"] = city_df["Miasto"].str.split(", ", expand = True)[0]
city_df = insert_id_col(city_df, "ID_miasta")

city_df.to_csv(csv_tables_dir_path/"Miasta_db.csv",
                       index = False, sep = ";", columns = ["ID_miasta","Miasto"])



# Atrakcje do tabeli csv.
attractions_only: pd.DataFrame = insert_id_col(insert_id_col(pd.DataFrame(all_info_trip["Atrakcja"].str.split(pat =", ")),
                                               "id_miasta").explode("Atrakcja"), "id_atrakcji")


attractions_only.to_csv(csv_tables_dir_path/"Atrakcje_db.csv",
                       index = False, sep = ";")

### Tworzenie tabeli rodzaje_wycieczek.

In [90]:
trips_type:pd.DataFrame = all_info_trip[["Matematyk","Cena"]]
trips_type = insert_id_col(trips_type, "id_miasta")
trips_type = insert_id_col(trips_type, "ID_rodzaj_wycieczki")


trips_type.to_csv(csv_tables_dir_path/"Rodzaje_wycieczek_db.csv",
                       index = False, sep = ";")